In [ ]:
from model.u2net_delta import U2NET
# from torchsummary import summary
import torch, os, cv2
from typing import List
from cv2 import Mat
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from torchvision import transforms
from data_loader import RescaleT
from data_loader import ToTensor
from data_loader import ToTensorLab
from u2net_test import save_output, normPRED

In [ ]:
model = U2NET().to('cuda')
model.load_state_dict(torch.load('saved_models/u2net/u2net.pth'))
model.process_filters()

In [ ]:
class DataFrames(Dataset):
    def __init__(self, dir: str, paths: List[str], transform=None, target_transform=None) -> None:
        super().__init__()
        self.img_paths: List[str] = paths
        self.dir: str = dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) -> int:
        return len(self.img_paths)

    def __getitem__(self, idx: int) -> Mat:
        img_path = os.path.join(self.dir, self.img_paths[idx])
        image = cv2.imread(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image

In [ ]:
paths = os.listdir('test_data')
dataset = DataFrames('test_data', paths, transform=transforms.Compose([RescaleT(320), ToTensorLab(flag=0)]))
dataloader = DataLoader(dataset, 1)

In [ ]:
for i, img in tqdm(enumerate(dataloader)):
    img = img.to('cuda').float()
    d1,d2,d3,d4,d5,d6,d7= model(img)
    # normalization
    pred = d1[:,0,:,:]
    pred = normPRED(pred)

    # save results to test_results folder
    if not os.path.exists('output'):
        os.makedirs('output', exist_ok=True)
    save_output(paths[i], pred, 'output')

    del d1,d2,d3,d4,d5,d6,d7

In [ ]:
t = torch.rand((1, 3, 144, 144)).to('cuda')
for i in tqdm(range(20)):
    out = model(t)